In [1]:
# Librerias
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask
from shapely.geometry import box
from joblib import load
from rasterio.features import shapes
from shapely.geometry import shape

Es importante tener en cuenta que a este punto se debe contar con una capa de extensión ".shp" en donde se han creado y clasificado polígonos de interés, para este caso: Bosque, No Bosque y Sin Información (o nubes). Esta manera permite una clasificación sencilla y bastante eficaz. 

In [ ]:
# === 1. Lista de imágenes empleadas en el entrenamiento ===
# Estas son las imágenes con las cuales se realizó el shape "entranimiento.shp" en donde se crearon clases
# para este caso en la tabla de atributos existe "class" donde: 
# 0:= Bosque
# 1:= No Bosque
# 3:= Sin información

tif_paths = [
    "Entrenamiento/L15-0603E-1038N_clip_4.tif",
    "Entrenamiento/L15-0603E-1039N_clip_4.tif",
    "Entrenamiento/L15-0603E-1040N_clip_4.tif"
]

# === 2. Cargar shapefile con polígonos y etiquetas ===
shapefile_path = "Entrenamiento/entrenamiento.shp"
gdf = gpd.read_file(shapefile_path)

# === 3. Inicializar contenedores de entrenamiento ===
X_train = []
y_train = []

# === 4. Iterar sobre cada imagen ===
for tif_path in tif_paths:
    with rasterio.open(tif_path) as src:
        transform = src.transform
        crs = src.crs
        profile = src.profile
        img_data = src.read()  # (bands, height, width)
        height, width = img_data.shape[1:]

        # Convertir a CRS del raster
        gdf_proj = gdf.to_crs(crs)

        # Filtrar solo los polígonos que se solapan con esta imagen
        bounds = src.bounds
        gdf_local = gdf_proj[gdf_proj.intersects(gpd.GeoSeries([box(*bounds)], crs=crs)[0])]

        # Saltar si no hay polígonos para esta imagen
        if gdf_local.empty:
            continue

        # Extraer muestras para cada polígono dentro del raster
        for _, row in gdf_local.iterrows():
            geometry = [row['geometry']]
            label = row['class']

            try:
                out_image, _ = mask(dataset=src, shapes=geometry, crop=True)
                out_image = out_image.transpose(1, 2, 0)  # (rows, cols, bands)

                # Quitar nodata
                valid_pixels = out_image.reshape(-1, out_image.shape[2])
                valid_pixels = valid_pixels[~np.any(valid_pixels == profile['nodata'], axis=1)]

                X_train.extend(valid_pixels)
                y_train.extend([label] * len(valid_pixels))

            except ValueError:
                # El polígono no intersectó bien, lo omitimos
                continue

# Convertir a arreglos numpy
X_train = np.array(X_train)
y_train = np.array(y_train)

print(f"Entrenamiento final: {X_train.shape[0]} muestras de {X_train.shape[1]} bandas.")

from sklearn.ensemble import RandomForestClassifier
from joblib import dump

# Entrenar modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)


# Guardar modelo entrenado
# El apredizaje se extrae en formato ".joblib" de manera que sea aplicable a otras imagenes diferentes
modelo='MathisiBeta'
dump(clf, fr"{modelo}.joblib")
print(fr"Modelo guardado como '{modelo}.joblib'")